<a href="https://colab.research.google.com/github/Fabihalt/PDS1920/blob/master/FishClassfierLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Image Classifer trainieren**#

In diesem Notebook wird ein Image Classfier zur Fischerkennung auf Bildern trainiert. Die Basis ist ein ResNet50-Netzes (wie in der Vorlesung vorgestellt), dass mit den von Kaggle zur Verfügung gestellten Daten Trainingsdaten anglernt wird. Im späteren Verlauf wir diesem trainnierten Classifier die von unserem Object Detector gecroppten Bilder zur Bestimmung der Fischarten übergeben.

##**Vorbereitung**##

In [0]:
# Einbinden aller benoetigten Packete
from fastai import *
from fastai.vision import *

In [0]:
# Google Drive mounten
# Daten aus dem Drive (Achtung Account!)
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Directory aendern um auf Ordner zuzugreifen
cd drive/My Drive/kaggle competition
# Pfad pruefen
path = Path('data/train')
path.ls()
#path = Path('')

In [0]:
#DATABUNCH erstellen aus den bildern von kaggle
np.random.seed(42) 
data = ImageDataBunch.from_folder(path, train=".", 
                                  valid_pct=0.2,
                                  ds_tfms=get_transforms(do_flip=True), 
                                  size=128).normalize(imagenet_stats)

In [0]:
# Ueberpruefen ob alles geklappt hat
# Drei Dreierreihen mit Bildern anzeigen
data.show_batch(rows=3)
#data.classes, data.c, len(data.train_ds), len(data.valid_ds)

##**Learner erstellen (ResNet)**##

In [0]:
# #MODELING erstellen eines Learners mit resnet18 Architektur fuers erste (kompakt)
# # Metriken und learner

# # Import der Metriken
# from fastai.metrics import accuracy, top_k_accuracy

# #Top 3 Acc da viele Tunaarten dabei sind
# top_3_accuracy = partial(top_k_accuracy, k=3)
# metriken = [accuracy,top_3_accuracy]
# # Learner mit den Metriken erstellen
# learn = cnn_learner(data, models.resnet18, metrics=metriken)

In [0]:
#MODELING erstellen eines Learners mit resnet34 Architektur als zweite Variante (kompakt)
# Metriken und learner

## Import der Metriken
#from fastai.metrics import accuracy, top_k_accuracy

## Top 3 Acc da viele Tunaarten dabei sind
#top_3_accuracy = partial(top_k_accuracy, k=3)
#metriken = [accuracy,top_3_accuracy]
## Learner mit den Metriken erstellen
#learn = cnn_learner(data, models.resnet34, metrics=metriken)

In [0]:
#MODELING erstellen eines Learners mit resnet50 Architektur als aufwendigste Variante

# Metriken und learner
# Import der Metriken
from fastai.metrics import accuracy, top_k_accuracy

# Top 3 Acc da viele Tunaarten dabei sind
top_3_accuracy = partial(top_k_accuracy, k=3)
metriken = [accuracy,top_3_accuracy]

# Learner mit den Metriken erstellen
learn = cnn_learner(data, models.resnet50, metrics=metriken)

##**Training**##
Das Training findet nach der aus der Vorlesung und aus dem fast.ai-Kurs bekannten Methodik statt. Es baut Hauptsächlich auf der korrekten Learningrate und der richtigen Anzahl von Epochen auf. Desweiteren wird zunächst nur die Letzten Layer des Netztes traininert da es sich um ein vortrainiertes Netz handelt, erst im Anschluss werden die restlichen Layer für das training zugänglich gemacht (unfreezing) und entsprechend ebenfalls trainiert.

###Erste Stufe###

In [0]:
# Optimale Learningrate (Lernrate) finden
learn.lr_find() 
# Plotten der LR
learn.recorder.plot(suggestion=True)

In [0]:
# Erste Stufe des Trainings
# trainieren mit entsprechender lr
learn.fit_one_cycle(5, max_lr=1E-02)

In [0]:
# Speicher des Zwischenstandes
# (wir merken uns, dass wir bisher auf einer Bildgroesse von 128 traininieren)
learn.save('stage-1-128')
# Wir koennen immer wieder zureuck auf den alten Stand falls wir uns verschlechtern

In [0]:
# Sofern noetig trail and error ausprobieren der LR und Epochenzahl um optimales 
# Ergebniss zu finden
#learn.fit_one_cycle(7, max_lr=1E-03)
#learn.load('stage-1-128')
#learn.fit_one_cycle(12, max_lr=1E-03)
#learn.load('stage-1-128)

###Zweite Stufe###

In [0]:
# Nach erfolgreicher Erster Stufe
# unfreezen und lr nochmals finden
learn.unfreeze()
learn.lr_find()
# Plotten
learn.recorder.plot(suggestion=True)

In [0]:
# trainieren mit slice
# Wie aus dem Kurs bekannt, wird das gesamte Netz nun mit einer slice LR trainiert
# Der Unterschied zw. erster und zweter LR sollte mindestens 1:10 betragen 
learn.fit_one_cycle(8, max_lr=slice(1e-05,1e-04))

In [0]:
# Speichern des Lernerfolges aus der zweiten Stufe des Trainings
learn.save('stage-2-128')

###Dritte Stufe###
Sofern benoetigt wird hier das training des Netztes fortgesetzt, indem der selbe Datensatz mit hoeheren Bildgrößen an das Netz zum trainieren weitergegeben wird. Die ist ein Trick der als eine art Feintuning zu sehen ist.

In [0]:
# Laden des Learners
#learn.load('stage-2-128')

In [0]:
#DATABUNCH mit veränderter groesse Uebergeben
#Ebenfalls kann die Batchsize veraendert werden (kleiner)

In [0]:
# LR und Plotten

In [0]:
# Trainineren

In [0]:
# Speichern falls Verbesserung eingetreten ist
learn.save('stage-3-224')

###Evaluation des aktuellen Models###

In [0]:
# Qualitative Evaluation 
# HEATMAP
# Visuelle Interpretation des Outputs (hier top loss)
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)

# 
# plot_top_losses mit heatmap um Grad-CAM anzuzeigen
interp.plot_top_losses(25,figsize=(21,21),heatmap=True)

###Exportieren des CNN für die spätere Verwendung###

In [0]:
# Exportieren des aktuellen Models
learn.export(file='fish_IC_Model.pkl')